In [249]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# from tensorflow.keras.metrics import categorical_accuracy
from sklearn.metrics import accuracy_score

In [250]:
# CUDA 를 이용한 GPU 가속
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))

# Mac M1에서 Pytorch 를 이용할 때 GPU 연산 가속을 위한 MPS
elif torch.torch.backends.mps.is_available():
    device = torch.device('mps:0')

    print("MPS enabled.")

# CPU 이용
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

MPS enabled.


In [251]:
CFG = {
    'EPOCHS':30,
    'LEARNING_RATE':8e-3,
    'BATCH_SIZE':16,
    'SEED':42
}

In [252]:
torch.manual_seed(CFG['SEED'])
np.random.seed(CFG['SEED'])
random.seed(CFG['SEED'])

In [253]:
# IRIS dataset 불러오기
iris = load_iris()
features = iris.data
labels = iris.target

In [254]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [255]:
df['target'].value_counts()

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64

In [256]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=CFG['SEED'])
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=CFG['SEED'])

In [257]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, index):
        x = torch.tensor(self.features[index], dtype=torch.float32)
        y = torch.tensor(self.labels[index], dtype=torch.long)
        return x, y

    def __len__(self):
        return len(self.features)

In [258]:
train_dataset = CustomDataset(x_train, y_train)
val_dataset = CustomDataset(x_val, y_val)
test_dataset = CustomDataset(x_test, y_test)

In [259]:
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'])
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'])

In [260]:
class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        self.input_layer    = nn.Linear(input_dim, 512)
        self.hidden_layer1    = nn.Linear(512, 256)
        self.hidden_layer2    = nn.Linear(256, 128)
        self.hidden_layer3  = nn.Linear(128, 64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()


    def forward(self,x):
        out = self.input_layer(x)
        out = self.relu(out)
        out = self.hidden_layer1(out)
        out = self.relu(out)
        out = self.hidden_layer2(out)
        out = self.relu(out)
        out = self.hidden_layer3(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [261]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_val_loss = float('inf')
    best_model_state_dict = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = []

        val_loss = []
        preds, trues = [], []

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        model.eval()

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                logit = model(inputs)
                loss = criterion(logit, labels)

                val_loss.append(loss.item())

                preds += logit.argmax(1).detach().cpu().numpy().tolist()
                trues += labels.detach().cpu().numpy().tolist()

            _val_loss = np.mean(val_loss)

        # _val_categorical_accuracy = np.mean(categorical_accuracy(trues, preds))
        _val_accuracy = accuracy_score(trues, preds)
        _train_loss = np.mean(train_loss)

        print(f"Epoch {epoch+1}: "
              f"Train Loss: {_train_loss:.4f}, "
              f"Val Loss: {_val_loss:.4f}, "
              f"Val Accuracy: {_val_accuracy:.4f}")

        # Save the best model
        if _val_loss < best_val_loss:
            best_val_loss = _val_loss
            best_model_state_dict = model.state_dict()

    model.load_state_dict(best_model_state_dict)
    return model

In [262]:
def test(model, test_loader, criterion):
    test_loss = 0.0
    accuracy = 0.0
    num_samples = len(test_loader.dataset)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            accuracy += (predicted == labels).sum().item()

    avg_accuracy = accuracy / num_samples * 100

    print(f"Test Accuracy: {avg_accuracy:.2f}%")

In [263]:
model = NeuralNetworkClassificationModel(4, 3)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=CFG['LEARNING_RATE'], momentum=0.9)

train(model, train_loader, val_loader, criterion, optimizer, CFG['EPOCHS'])

Epoch 1: Train Loss: 1.1033, Val Loss: 1.0821, Val Accuracy: 0.3000
Epoch 2: Train Loss: 1.0603, Val Loss: 1.0393, Val Accuracy: 0.5667
Epoch 3: Train Loss: 0.9898, Val Loss: 0.9948, Val Accuracy: 0.5667
Epoch 4: Train Loss: 0.8825, Val Loss: 0.8631, Val Accuracy: 0.5667
Epoch 5: Train Loss: 0.6794, Val Loss: 0.6324, Val Accuracy: 0.5667
Epoch 6: Train Loss: 0.4894, Val Loss: 0.4776, Val Accuracy: 0.8667
Epoch 7: Train Loss: 0.4107, Val Loss: 0.4477, Val Accuracy: 0.7000
Epoch 8: Train Loss: 0.3289, Val Loss: 0.4197, Val Accuracy: 0.7333
Epoch 9: Train Loss: 0.2446, Val Loss: 0.6397, Val Accuracy: 0.6000
Epoch 10: Train Loss: 0.2136, Val Loss: 0.2865, Val Accuracy: 0.8667
Epoch 11: Train Loss: 0.2576, Val Loss: 0.6028, Val Accuracy: 0.7333
Epoch 12: Train Loss: 0.4211, Val Loss: 0.5837, Val Accuracy: 0.7333
Epoch 13: Train Loss: 0.2865, Val Loss: 0.2027, Val Accuracy: 0.9333
Epoch 14: Train Loss: 0.5163, Val Loss: 0.9919, Val Accuracy: 0.5667
Epoch 15: Train Loss: 0.5363, Val Loss: 0.5

NeuralNetworkClassificationModel(
  (input_layer): Linear(in_features=4, out_features=512, bias=True)
  (hidden_layer1): Linear(in_features=512, out_features=256, bias=True)
  (hidden_layer2): Linear(in_features=256, out_features=128, bias=True)
  (hidden_layer3): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
)

In [264]:
test(model, test_loader, criterion)

Test Accuracy: 96.67%
